# Intento 3

Este intento es similar al anterior con la única diferencia de que el *frame size* empleado es (96,96) en lugar de (64,64). Además emos usado una tasa de *dropout* algo mayor (del 50% en lugar del 30%)

## 0. Descarga del dataset

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

train_ds_path  ='C:/Users/Usuario/Documents/Master/Aprendizaje Profundo/Udemy/deeplearning-az/datasets/Part 2 - Convolutional Neural Networks (CNN)/dataset/training_set'
test_ds_path   ='C:/Users/Usuario/Documents/Master/Aprendizaje Profundo/Udemy/deeplearning-az/datasets/Part 2 - Convolutional Neural Networks (CNN)/dataset/test_set'
cat_or_dog_path='C:/Users/Usuario/Documents/Master/Aprendizaje Profundo/Udemy/deeplearning-az/datasets/Part 2 - Convolutional Neural Networks (CNN)/dataset/single_prediction/cat_or_dog_1.jpg'


#train_ds_path  ='.\\data\\training_set'
#test_ds_path   ='.\\data\\test_set'
#cat_or_dog_path='.\\data\\single_prediction\\cat_or_dog_1.jpg'


Primero, importar las librerías y paquetes

In [2]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import BatchNormalization
from keras.layers import GlobalAveragePooling2D
from keras.layers import Dropout
# Nota, algunas capas no están importadas aquí y se importan directamente en el código con tf.keras.laters.[CAPA]

import matplotlib.pyplot as plt
import tensorflow as tf
import os
import numpy as np
import random as rn

Fijamos seeds para poder reproducir resultados (aunque aun así a veces no lo conseguimos, probablementa haya inicializaciones que no dependan de estas seeds)

In [3]:
os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(42)
rn.seed(12345)
tf.random.set_seed(1234)

## 1. Construcción del modelo CNN añadiendo un tamaño de imagen mayor


El tamaño de imagen que emplearemos será de 96x96, y el dropout rate es del 30%

In [4]:
frame_size = (96, 96)
"""
esta dupla nos permitirá parametrizar la resolución
de entrada de las imágenes
"""

def crear_clasificador_intento2():
    classifier = Sequential()
    classifier.add(Conv2D(filters = 32,kernel_size = (3, 3), 
                      input_shape = (*frame_size, 3), activation = "relu"))
    classifier.add(MaxPooling2D(pool_size = (2,2)))
    classifier.add(Conv2D(filters = 32,kernel_size = (3, 3), activation = "relu"))
    classifier.add(MaxPooling2D(pool_size = (2,2)))
    classifier.add(Flatten())
    classifier.add(Dense(units = 128, activation = "relu"))
    classifier.add(Dropout(0.3))
    classifier.add(Dense(units = 1, activation = "sigmoid"))    
    return classifier

## 2. Entrenamiento del intento 2

En primer lugar instanciamos nuestro modelo y compilamos usando:
* Un optimizador Adam. La learning rate que emplea por defecto es 0.001
* Binary cross entropy como función de coste a minimizar.  

In [5]:
classifier = crear_clasificador_intento2()
classifier.compile(optimizer = "adam", 
                   loss = "binary_crossentropy", 
                   metrics = ["accuracy"])
classifier.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 94, 94, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 47, 47, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 45, 45, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 22, 22, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 15488)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               1982592   
_________________________________________________________________
dropout (Dropout)            (None, 128)               0

En segundo lugar, generamos los datasets de entrenamiento y test. Emplearemos un tamaño de batch de 32

In [6]:
from keras.preprocessing.image import ImageDataGenerator

batch_size=32 

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

training_dataset = train_datagen.flow_from_directory(train_ds_path,
                                                     target_size=frame_size,
                                                     batch_size=batch_size,
                                                     class_mode='binary')

testing_dataset = test_datagen.flow_from_directory(test_ds_path,
                                                   target_size=frame_size,                                                   
                                                   batch_size=batch_size,
                                                   class_mode='binary')

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


Definimos el callback y realizamos el entrenamiento con las condiciones descritas en la sección de introducción.

In [7]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta = 0, patience=5),
]                                                   
                                                   
history1 = classifier.fit(x=training_dataset,
                         steps_per_epoch=8000/batch_size,
                         epochs=100,
                         validation_data=testing_dataset,
                         validation_steps=2000/batch_size,
                         workers=4) # "Si pedimos más de un proceso el rendimiento mejora un poco


Epoch 1/100
250/250 [==============================] - 31s 123ms/step - loss: 0.6808 - accuracy: 0.5705 - val_loss: 0.6015 - val_accuracy: 0.6755
Epoch 2/100
250/250 [==============================] - 31s 124ms/step - loss: 0.5921 - accuracy: 0.6837 - val_loss: 0.5527 - val_accuracy: 0.7220
Epoch 3/100
250/250 [==============================] - 30s 120ms/step - loss: 0.5401 - accuracy: 0.7212 - val_loss: 0.5366 - val_accuracy: 0.7390
Epoch 4/100
250/250 [==============================] - 36s 144ms/step - loss: 0.5216 - accuracy: 0.7476 - val_loss: 0.4993 - val_accuracy: 0.7560
Epoch 5/100
250/250 [==============================] - 48s 191ms/step - loss: 0.4813 - accuracy: 0.7595 - val_loss: 0.4875 - val_accuracy: 0.7605
Epoch 6/100
250/250 [==============================] - 51s 202ms/step - loss: 0.4782 - accuracy: 0.7798 - val_loss: 0.5000 - val_accuracy: 0.7655
Epoch 7/100
250/250 [==============================] - 52s 206ms/step - loss: 0.4559 - accuracy: 0.7773 - val_loss: 0.4705 -

250/250 [==============================] - 31s 122ms/step - loss: 0.1006 - accuracy: 0.9606 - val_loss: 0.7995 - val_accuracy: 0.8090
Epoch 57/100
250/250 [==============================] - 30s 121ms/step - loss: 0.1011 - accuracy: 0.9592 - val_loss: 0.7548 - val_accuracy: 0.8260
Epoch 58/100
250/250 [==============================] - 30s 119ms/step - loss: 0.1106 - accuracy: 0.9574 - val_loss: 0.8070 - val_accuracy: 0.8180
Epoch 59/100
250/250 [==============================] - 30s 120ms/step - loss: 0.1029 - accuracy: 0.9596 - val_loss: 0.7569 - val_accuracy: 0.8225
Epoch 60/100
250/250 [==============================] - 30s 120ms/step - loss: 0.0976 - accuracy: 0.9592 - val_loss: 0.7856 - val_accuracy: 0.8130
Epoch 61/100
250/250 [==============================] - 30s 119ms/step - loss: 0.1067 - accuracy: 0.9613 - val_loss: 0.7978 - val_accuracy: 0.8155
Epoch 62/100
250/250 [==============================] - 30s 120ms/step - loss: 0.1011 - accuracy: 0.9649 - val_loss: 0.8525 - val_a

Ploteamos el resultado

In [8]:
def plot_resultados_training(history):
    fig, axes = plt.subplots(1,2, figsize=(18,6))
    axes[0].plot(history.history['accuracy'], label='Train')
    axes[0].plot(history.history['val_accuracy'], label='Validation')
    axes[0].legend()
    axes[0].set_title('Accuracy')
    axes[1].plot(history.history['loss'], label='Train')
    axes[1].plot(history.history['val_loss'], label='Validation')
    axes[1].set_title('Cross entropy')
plot_resultados_training(history)

NameError: name 'history' is not defined

In [ ]:
classifier.save('.\\models\\clasificador3')

## Comentario
La divergencia entre las precisiones de entrenamiento y validación ha empeorado. Aunque cabe destacar que la red ha mejorado su tasa de precisión
en entrenamiento, seguramente debido a que las imágenes aportan más información útil.

## Propuesta de mejora
Seguiremos aumentando el parámetro del índice de olvido hasta su máximo teórico de 0.5 para ver si esto mejora el sobre-entrenamiento.